In [2]:
import xtrack as xt
import xobjects as xo
import xpart as xp
from scipy import constants 
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

line_LEIR = xt.Line.from_json('../../leir.json')
twiss = line_LEIR.twiss()
particle_ref=line_LEIR.particle_ref

particle_ref=line_LEIR.particle_ref
mass0=particle_ref.mass0
gamma0=particle_ref.gamma0
beta0=particle_ref.beta0
p0c=particle_ref.p0c

circumference = twiss.circumference #m

clight=constants.speed_of_light
T_per_turn = circumference/(clight*beta0)

# optics at e-cooler (approximate), in m
beta_x=5
beta_y=5
dx=0

qx = twiss.qx       #0.8200408678 #1.82
qy = twiss.qy       #0.7200210637 #2.72
dqx= twiss.dqx      #-0.0005113579791
dqy= twiss.dqy      #-0.0001665178543
qs = twiss.qs
bets0=twiss.bets0

arc = xt.LineSegmentMap(
        qx=qx, qy=qy,
        length=circumference,
        betx=beta_x,
        bety=beta_y,
        dx=dx)

###############################

arc_matching = xt.LineSegmentMap(
        qx=qx, qy=qy,
        length=circumference,
        betx=beta_x,
        bety=beta_y,
        dx=dx,
        qs=qs,
        bets=bets0)

line_matching=xt.Line([arc_matching])
line_matching.build_tracker()

num_particles=int(1e4)

#sigma_dp = 2e-3 # Standard LEIR value
sigma_dp = 5e-3 # from Schotttky: 5e-3
 
gemitt_x = 14e-6
gemitt_y = 14e-6

nemitt_x = gemitt_x*beta0*gamma0
nemitt_y = gemitt_y*beta0*gamma0

particles = xp.generate_matched_gaussian_bunch(
        num_particles=num_particles,
        # total_intensity_particles=bunch_intensity,
        nemitt_x=nemitt_x, nemitt_y=nemitt_y, sigma_z=4.2,
        particle_ref=particle_ref,
        line=line_matching,        
        )

# create desired beam
bunch_intensity = None
sigma_z=-bets0*sigma_dp

particles.delta = np.random.normal(loc=0.0, scale=sigma_dp, size=num_particles)
#particles.zeta = 0
particles.zeta = np.random.uniform(-circumference/2, circumference/2, num_particles)
particles0=particles.copy()

# simulation parameters: simulate 1 s of cooling, and take data once every 10 ms
max_time_s = 1
int_time_s = 1*1e-3

# compute length of simulation, as well as sample interval, in turns
num_turns = int((max_time_s / T_per_turn).item())
save_interval = int((int_time_s / T_per_turn).item())


# create a monitor object, to reduce holded data
monitor = xt.ParticlesMonitor(start_at_turn=0, stop_at_turn=1,
                        n_repetitions=int(num_turns/save_interval),
                        repetition_period=save_interval,
                        num_particles=len(particles.x))


current = 0.6  # amperes
cooler_length = 2.5  # m cooler length
radius_e_beam = 25 * 1e-3
temp_perp = 100e-3 # <E> [eV] = kb*T
temp_long =  1e-3 # <E> [eV]
magnetic_field = 0.075  # T for LEIR
magnetic_field_ratio = 0

# current=0.6 #amperes
# cooler_length = 3 # m cooler length
# r_beam=25*1e-3

# T_perp = 0.01 # <E> [eV] = kb*T
# T_l =  0.001 # <E> [eV]
# magnetic_field = 0.07 # T for LEIR
# B_ratio=0

electron_cooler = xt.ElectronCooler(current=current,
                                        length=cooler_length,
                                        radius_e_beam=radius_e_beam,
                                        temp_perp=temp_perp, temp_long=temp_long,
                                        magnetic_field=magnetic_field)

line = xt.Line(elements=[monitor, electron_cooler, arc],element_names=['monitor','electron_cooler','arc'])
                                
line.particle_ref = particle_ref
context = xo.ContextCpu(omp_num_threads=4)
line.build_tracker(_context=context)

charge_list = [40, 45, 50]  # e

for charge in tqdm(charge_list):
    particles=particles0.copy()
    particles.q0=charge
    #particles.delta = np.random.normal(loc=0.0, scale=sigma_dp_loop, size=num_particles)
    # particles_emittance=particles_emittance0.copy()    

    # just track all particles, and keep turn-by-turn data (memory expensive!)
    line.track(particles, num_turns=num_turns,
            turn_by_turn_monitor=False,with_progress=True)

    # extract relevant values
    x = monitor.x[:,:,0]
    px = monitor.px[:,:,0]
    y = monitor.y[:,:,0]
    py = monitor.py[:,:,0]
    delta = monitor.delta[:,:,0]
    zeta = monitor.zeta[:,:,0]
    time = monitor.at_turn[:, 0, 0] * T_per_turn
    
    # compute actions. for x, remove the dp/p contribution:
    action_x = ((x-dx*delta)**2/beta_x + beta_x*px**2)
    # for y, simple compute:
    action_y = (y**2/beta_y + beta_y*py**2)
    emittance_x=np.mean(action_x, axis=1)/2
    norm_emittance_x=np.mean(action_x,axis=1)/2*gamma0*beta0
    norm_emittance_y=np.mean(action_y,axis=1)/2*gamma0*beta0

    geo_emittance_x=np.mean(action_x,axis=1)/2
    geo_emittance_y=np.mean(action_y,axis=1)/2

    # sigma_x_all_angles = np.array([np.sqrt(beta_x * geo_emittance_x) for angle in angle_list])
    # sigma_x_95 = np.percentile(sigma_x_all_angles, 95, axis=0)
    p0c = particle_ref.p0c
    momentum = p0c*delta+p0c                

    # Save results
    filepath = f'results/emittance_xsuite_Q={charge}.npz'
    np.savez(filepath, time=time,
             emittance_x=geo_emittance_x, emittance_y=geo_emittance_y,
             dp_p=delta)


Xcoll not installed


Loading line from dict:   0%|          | 0/2526 [00:00<?, ?it/s]

Done loading line from dict.           
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


  0%|          | 0/3 [00:00<?, ?it/s]

Tracking:   0%|          | 0/361300 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [03:26<06:52, 206.23s/it]

Tracking:   0%|          | 0/361300 [00:00<?, ?it/s]

 67%|██████▋   | 2/3 [06:27<03:11, 191.49s/it]

Tracking:   0%|          | 0/361300 [00:00<?, ?it/s]

100%|██████████| 3/3 [09:52<00:00, 197.37s/it]
